In [1]:
#here we used free library for training the data
import cv2
import numpy as np
import os

# Load OpenCV's built-in face detector
#here we import cascadeclassifier the with  help of cv2
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# make dataset directory 
dataset_path = "face_dataset"
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)        #create the dataset directory it does not exist

name = input("Enter your name: ")
person_path = os.path.join(dataset_path, name)
os.makedirs(person_path, exist_ok=True)    # create a directory for the person if it does not exist 

cap = cv2.VideoCapture(0)      #open the web camera 
count = 0   # intialize the image count

while count < 100:  # Collect 100 face images
    ret, frame = cap.read()     # read a frame from the webcam
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #convert the frame to gray scale
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)  #detect face from the frame 

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]   # extract the face region from the gray scale frame
        face = cv2.resize(face, (100, 100))  #resize the face to 100*100 pixel
        cv2.imwrite(f"{person_path}/{count}.jpg", face) # save the face image to the person directory
        count += 1 # increment the image count
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  #draw a rectangle around the face 

    cv2.imshow("Face Collection", frame) # display the frame with detected the face 
    if cv2.waitKey(1) & 0xFF == ord("q"): # exit the loop if q is press 
        break

cap.release() # release the webcam
cv2.destroyAllWindows() # close all open cv windows


Enter your name:  Akhilesh PAndey


# Train the KNN Model

In [2]:
import cv2
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier  # here all type of algorithm  is used like svm,knn etc in sklearn
import pickle

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

dataset_path = "face_dataset"
data = []        # list to store image the data
labels =[]         # list to store labels correspoing to each image 
names = [name for name in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, name))]  # Filter out non-directory entries

for idx, name in enumerate(names):
    person_path = os.path.join(dataset_path, name)
    for file in os.listdir(person_path):
        file_path = os.path.join(person_path, file)
        if os.path.isfile(file_path):  # Ensure it's a file
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) # read the image in gray scale 
            if img is not None:
                img = cv2.resize(img, (100, 100))
                data.append(img.flatten())  # Convert to 1D array
                labels.append(idx)          # add label index corresponing to the person 
            else:
                print(f"Failed to load image: {file}")

data = np.array(data)    # convert the data list to numpy array
labels = np.array(labels) # convert label list to numpy array

# Train KNN model
knn = KNeighborsClassifier(n_neighbors=5)   # react knn classifier with 5 neighbour
knn.fit(data, labels)  # train the classfier with image,data and label

# Save the model
with open("face_recognition_model.pkl", "wb") as f:
    pickle.dump((knn, names), f)    # save the model and name to a file  

print("Training complete and model saved!")


Training complete and model saved!


# Real Time Face Recognition 

In [ ]:
import cv2
import numpy as np
import pickle
import csv
import datetime
import os
# Load face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load trained model
with open("face_recognition_model.pkl", "rb") as f:
    knn, names = pickle.load(f)      #load the knn model name from the pickle file 

cap = cv2.VideoCapture(0)

# Attendance file
attendance_file = "attendance.csv"     # make the csv file for attendance 

# Check if file exists, else create it
if not os.path.exists(attendance_file):
    with open(attendance_file, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Name", "Date", "Time"])

marked_names = set()  # Track marked names to avoid duplicate marking

while True:
    ret, frame = cap.read()    # read the frame from the webcam
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (100, 100))
        face = face.flatten().reshape(1, -1)  # Prepare for KNN and convert to one d array and reshape  

        label = knn.predict(face)[0] # pedict the label of the face using the knn model
        name = names[label] # give the name correspoding to the pridicated the label

        # Display name
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2) # display the name above the face

        # Mark attendance if not already marked
        if name not in marked_names:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # this is storing year,month and day in acolumn and hour,min and second in other column
            with open(attendance_file, "a", newline="") as file:
                writer = csv.writer(file)
                writer.writerow([name, timestamp.split()[0], timestamp.split()[1]])

            print(f"Attendance marked for {name}")
            marked_names.add(name)

    cv2.imshow("Attendance System", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Attendance marked for ritik
